In [3]:
import numpy as np
import math
import random
import os

def READFILE(loc, n):
	F = open(loc, "r")
	F.readline()
	inputs = []
	keys = []
	outputs = []
	variables = {}
	for i in range(n-1):
		A = F.readline()
		B = A
		# L = A.split()
		if A[:5] == "INPUT":
			if A[:14] == "INPUT(keyinput":
				keys.append(B.replace("(", " ").replace(")", " ").split()[1])
			else:
				inputs.append(B.replace("(", " ").replace(")", " ").split()[1])
		elif A[:6] == "OUTPUT":
			outputs.append(B.replace("(", " ").replace(")", "").split()[1])
		else:
			L = B.replace("(", " ").replace(")", " ").replace(",", "").split()
			T = []
			v = L[0]
			T = L[2:]
			
			variables[v] = T
	return inputs, keys, outputs, variables

def CALCULATE(gates, vals, gate):
	L = gates[gate]
	# print( gate, L)
	if L[0] == "buf":
		return int(vals[L[1]])

	elif L[0] == "not":
		return int(not(vals[L[1]]))
	
	elif L[0] == "and":
		# return int(vals[L[1]] and vals[L[2]])
		t = vals[L[1]]
		for i in range(2, len(L)):
			t = int(t and vals[L[i]])
		return int(t)
	
	elif L[0] == "nand":
		# return int(not (vals[L[1]] and vals[L[2]]))
		t = vals[L[1]]
		for i in range(2, len(L)):
			t = int(t and vals[L[i]])
		return int(not t)
	
	elif L[0] == "or":
		# return int(vals[L[1]] or vals[L[2]])
		t = vals[L[1]]
		for i in range(2, len(L)):
			t = int(t or vals[L[i]])
		return int(t)
	
	elif L[0] == "nor":
		# return int(not (vals[L[1]] or vals[L[2]]))
		t = vals[L[1]]
		for i in range(2, len(L)):
			t = int(t or vals[L[i]])
		return int(not t)

	elif L[0] == "xor":
		# return int(vals[L[1]] ^ vals[L[2]])
		t = vals[L[1]]
		for i in range(2, len(L)):
			t = int(t ^ vals[L[i]])
		return int(t)

	elif L[0] == "xnor":
		# return int(not(vals[L[1]] ^ vals[L[2]]))
		t = vals[L[1]]
		for i in range(2, len(L)):
			t = int(t ^ vals[L[i]])
		return int(not t)

def NET(gates, values, loc, n):
	# inputs, keys, outputs, variables = READFILE(loc, n)
	L = list(gates.keys())
	L.sort()
	for i in L:
		# print(i)
		gates[i] = CALCULATE(gates, values, i)
		values[i] = gates[i]
	return values

# gives input-output pairs
def inp_out_pairs(inputs_given, keys_given):
	loc = "c432_enc10.bench"
	n = 239

	inp_vars, key_vars, out_vars, variables = READFILE(loc, n)

	inp_vars.sort()
	inputs = {}
	m = 0
	for i in inp_vars:
		inputs[i] = inputs_given[m]
		m = m+1

	key_vars.sort()
	keys = {}
	m = 0
	for i in key_vars:
		keys[i] = keys_given[m]
		m = m + 1

	gate_vals = {}
	for i in inputs:
		gate_vals[i] = inputs[i]

	for i in keys:
		gate_vals[i] = keys[i]

	gate_vals = NET(variables, gate_vals, loc, n)

	output = {}
	for i in out_vars:
		output[i] = gate_vals[i]

	M = list(output.keys())
	M.sort()

	out = []
	for i in M:
		out.append(output[i])

	return out

def circuit(inpt, key):
	return inp_out_pairs(inpt, key)

def original(inpt):
	K = [0,1,1,0,1,0,0,0,0,1,0,1,1,1,1,0]
	# 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0
	return inp_out_pairs(inpt, K)


def bit_coeff(inpt, key, output1, output2):
	A = np.random.randint(2, size = (inpt + key + output1 + output2))
	return A


def bit_next(L1, L2):
	n = len(L1)
	A = np.zeros(n)
	for i in range(n):
		p = random.random()
		if p < 0.5:
			A[i] = L1[i]
		else:
			A[i] = L2[i]
	return A

def update(C, texts):
	# T = np.argsort(C[:, 1])
	i = random.randint(0, len(C)-1)
	j = random.randint(0, len(C)-1)
	idx1 = i
	bit1 = C[idx1, 0]

	idx2 = j
	bit2 = C[idx2, 0]

	bit3 = bit_next(bit1, bit2)
	val = calculate(bit3, texts)

	if (abs(val) > abs(C[idx2, 1])):
		C[idx1, 0] = bit2
		C[idx1, 1] = C[idx2, 1]
		C[idx2, 0] = bit3
		C[idx2, 1] = val

	elif (abs(val) > abs(C[idx1, 1])):
		C[idx1, 0] = bit3
		C[idx1, 1] = val
	return C


def calculate(bits, texts):
	counter = 0
	n = len(texts)
	for i in range(n):
		val = 0
		for j in range(len(bits)):
			# print (bits[j], texts[i, j])
			val = val ^ (int(bits[j]) and texts[i, j])
		if(val == 0):
			counter += 1
	return counter - n/2.0


def generate_random(inpt, outpt, key, n):
	L = []
	for i in range(n):
		P = bit_coeff(inpt, key, 0, 0)
		x = circuit(P[:inpt], P[-key:])
		y = original(P[:inpt])
		L.append(np.concatenate((P, np.array(x), np.array(y))))
	return np.array(L)

def generate_key(inpt, outpt, key, KEY, n):

	L = []
	for i in range(n):
		M = bit_coeff(inpt, 0, 0, 0)
		P = np.concatenate((M, np.array(KEY)))
		x = circuit(P[:inpt], KEY)
		y = original(M)
		L.append(np.concatenate((P, np.array(x), np.array(y))))
	return np.array(L)

def genetic_update(inpt, outpt, key, n, t = 1000, itr = 10000):
	texts = generate_random(inpt, outpt, key, n)
	C = []
	for i in range(t):
		x = bit_coeff(inpt, key, outpt, outpt)
		print(i)
		counter = calculate(x, texts)
		C.append([x, counter])
	for i in range(itr):
		copy = np.array(C)
		C = update(copy, texts)
		
	# print(C)
	return np.array(C)


In [4]:

A = genetic_update(36, 7, 16, 1000)
print(A)


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27